In [4]:
import os
import sys
sys.path.insert(0, "/home/gstupp/projects/metaproteomics/")
#BASE = os.path.join(os.path.dirname(os.path.realpath(sys.argv[0])), "..")
BASE = "/home/gstupp/projects/Wolan/ana_extractions/analysis/H1"
DATA = os.path.join(BASE, "data")
OUT = os.path.join(BASE, "out")

from metaproteomics import utils
import pandas as pd
from itertools import chain
import re
import shelve
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from collections import Counter
from collections import defaultdict

from metaproteomics.analysis import build_loci ,functional_analysis
from metaproteomics.analysis.DBInfo import DBInfo

f = functional_analysis.Functionizer()

db_info = DBInfo("compil_mgm")
metadata = build_loci.read_metadata("metadata.csv")

In [7]:
metadata

,H1-PelletTriflic-Fl-R1,H1-PelletTriflic-Fl-R2,H1-PelletTriflic-Fl-R3,H1-SonicSupTriflic-V2-R1,H1-SonicSupTriflic-V2-R2,H1-SonicSupTriflic-V2-R3,H1-wholetriflic-R1,H1-wholetriflic-R2,H1-wholetriflic-R3
technical,1,2,3,1,2,3,1,2,3
sample_type,sonic_pellet,sonic_pellet,sonic_pellet,sonic_sup,sonic_sup,sonic_sup,whole,whole,whole
sample_name,H1-PelletTriflic-Fl-R1,H1-PelletTriflic-Fl-R2,H1-PelletTriflic-Fl-R3,H1-SonicSupTriflic-V2-R1,H1-SonicSupTriflic-V2-R2,H1-SonicSupTriflic-V2-R3,H1-wholetriflic-R1,H1-wholetriflic-R2,H1-wholetriflic-R3
path,/home/gstupp/projects/Wolan/ana_extractions/H1...,/home/gstupp/projects/Wolan/ana_extractions/H1...,/home/gstupp/projects/Wolan/ana_extractions/H1...,/home/gstupp/projects/Wolan/ana_extractions/H1...,/home/gstupp/projects/Wolan/ana_extractions/H1...,/home/gstupp/projects/Wolan/ana_extractions/H1...,/home/gstupp/projects/Wolan/ana_extractions/H1...,/home/gstupp/projects/Wolan/ana_extractions/H1...,/home/gstupp/projects/Wolan/ana_extractions/H1...


In [8]:
#%% Parse samples
samples = shelve.open(os.path.join(OUT,"samples.shelve"))
for sample_name, sample_info in tqdm(list(metadata.iteritems())):
    sample = build_loci.Sample(sample_name, sample_info.path, db_info, sample_info)
    samples[sample.sample_name] = sample

100%|██████████| 9/9 [01:32<00:00, 10.23s/it]


In [9]:
# Build protein clusters for each sample
protein_clusters = shelve.open(os.path.join(OUT,"protein_clusters.shelve"))
for name,sample in samples.items():
    protein_clusters[name] = sample.build_protein_clusters()

In [10]:
# generate proteins clusters across all samples
sample_pep_quant = {sample.sample_name:sample.pep_quant for sample in samples.values()}
grouped_loci = build_loci.group_across_samples(list(chain(*protein_clusters.values())), sample_pep_quant)
utils.save(grouped_loci, os.path.join(OUT,"grouped_loci.pkl.gz"), force=True)

load obo file /home/gstupp/projects/metaproteomics/metaproteomics/goatools/go-basic.obo
48437 nodes imported


In [11]:
# write out matrix
df = build_loci.to_df(grouped_loci, norm=False)
df.T.to_csv(os.path.join(OUT,"df.csv"))

In [12]:
df.shape

(9, 10606)

In [13]:
grouped_loci = utils.load(os.path.join(OUT,"grouped_loci.pkl.gz"))

In [14]:
# filter loci, min spectral count of 4 in at least two samples
min_quant = 4
min_samples = 2
grouped_loci = [locus for locus in grouped_loci if len([x for x in locus.quantification.values() if x>=min_quant]) >= min_samples]
len(grouped_loci)

8083

In [15]:
utils.save(grouped_loci, os.path.join(OUT,"grouped_loci_filt.pkl.gz"), force=True)

In [16]:
df = build_loci.to_df(grouped_loci, norm=False)
df.T.to_csv(os.path.join(OUT,"df_filt.csv"))
df.shape

(9, 8083)

In [17]:
# run functional annotation
for locus in tqdm(grouped_loci):
    locus.annotate()
utils.save(grouped_loci, os.path.join(OUT,"grouped_loci_filt_annot.pkl.gz"), force=True)

100%|██████████| 8083/8083 [59:31<00:00,  2.26it/s]


In [18]:
#%% mark as human/mouse and other
from metaproteomics.analysis import taxonomy
t = taxonomy.Taxonomy(host="wl-cmadmin.scripps.edu")
# human or mouse taxids and ancestors (up to the phylum chordata (7711)):
chordata = set(t.taxid_to_taxonomy(7711)['lineage'])
human = set(t.taxid_to_taxonomy(9606)['lineage']) - chordata
mouse = set(t.taxid_to_taxonomy(10090)['lineage']) - chordata
human_mouse = human | mouse
for locus in grouped_loci:
    locus.human_mouse = True if locus.lca in human_mouse else False

In [19]:
#%% Build a locus -> metadata table
for locus in grouped_loci:
    names = [x['d'] for x in locus.prot_info]
    gn=set(chain(*[re.findall("GN=([^ ]*)",name) for name in names]))
    locus.gn = ','.join(gn) if gn else ''
    locus.gn1 = list(gn)[0] if len(gn)==1 else ''
locus_df = pd.DataFrame({locus.cluster_id: {'name': locus.name, 'human_mouse': locus.human_mouse, 'lca':locus.lca, 'gn':locus.gn, 'gn1':locus.gn1} for locus in grouped_loci}).T
locus_df.to_csv(os.path.join(OUT,"locus_df.csv"))

In [5]:
grouped_loci = utils.load(os.path.join(OUT,"grouped_loci_filt_annot.pkl.gz"))

In [9]:
from collections import Counter

In [11]:
Counter([len(x.annotations.get('go',[]))==0 for x in grouped_loci])

Counter({False: 6018, True: 2065})

In [13]:
np.mean([len(x.annotations.get('go',[])) for x in grouped_loci])

2.5749103055796114